<a href="https://colab.research.google.com/github/leoIBM/Coursera_Capstone/blob/master/Week%203%20Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 This notebook will be mainly used for the capstone project.

**Week 2: Segmenting and Clustering Neighborhoods in Toronto**

Importing

In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from bs4 import BeautifulSoup # library to scrape information from web pages

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Downloading Data

In [36]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
print('Data downloaded!')
bs = BeautifulSoup(data, 'html.parser')

Data downloaded!


Using Beautiful Soul

In [21]:
data = []


for row in bs.findAll('tr'):
    lst = []
    for col in row.findAll('td'):
        lst.append(col.text.strip())
    if len(lst) == 3:
        data.append(lst)


Creating dataframe

In [22]:
pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(data)

df.head()

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Name the Columns

In [23]:

df.columns = ['Postalcode', 'Borough', 'Neighborhood']
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Drop Not assigned Boroughs

In [37]:
df_drop = df[df.Borough != "Not assigned"].reset_index(drop=True)
df_drop.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Grouping neighborhoods

In [29]:
df_group = df_drop.groupby(["Postalcode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_group.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Not assigned neighborhood, then the neighborhood same name as the borough.

In [33]:
for index, row in df_group.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
df_group.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


 print the number of rows of the dataframe

In [44]:
df_group.shape
print(df_group.shape)
print('103 rows')

(103, 3)
103 rows
